## TODO: 
- Predicciones para cardinalidades
- Capa de MySQL code validation?
- Como ordeno las tablas en orden? Ejemplo, si una tiene una FK de una tabla que todavía no se creó. Respuesta -> https://dba.stackexchange.com/questions/258248/table-creation-order dejas las FK y las PK para después de los create table asi no tenes problemas..

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.append("../")

from src.training_utils.training import load_model
from src.line_detection.hough import *
from ast import literal_eval
from src.ocr_utils.ocr import *

In [3]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from IPython.display import display
from PIL import Image
from torchvision import transforms as T
import torchvision
from paddleocr import PaddleOCR

In [4]:
PATH_MODELS = "/home/nacho/TFI-Cazcarra/data/models"
PATH_YOLO = "/home/nacho/TFI-Cazcarra/yolov3/runs/train"

## Realizo predicciones

In [5]:
img_path = '/home/nacho/TFI-Cazcarra/data/imagenes_diagramas/ERDiagramsMySQL-5.png'
img = Image.open(img_path).convert("RGB")
transform = T.Compose([T.ToTensor()])
img_tensor = transform(img)

In [6]:
def draw_bbox(img, xmin, ymin, xmax, ymax, score, label): 
    txt = get_class_name(label, le_dict) + ' ' + str(score)
    img = cv2.putText(img, txt, (int(xmin), int(ymin)),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255), 1)

    return cv2.rectangle(img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), 
                         (255,0,0), 1)

### 1. Extraigo las tablas

In [7]:
retinanet_tablas = load_model(os.path.join(PATH_MODELS, "model_best_tablas_retinanet.pt"))
faster_rcnn_tablas = load_model(os.path.join(PATH_MODELS, "model_best_tablas_fasterrcnn.pt"))

In [8]:
yolo_tablas = torch.hub.load('ultralytics/yolov5', 'custom', \
                             os.path.join(PATH_YOLO, "exp5", "weights", "best.pt"))

Using cache found in /home/nacho/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-2-8 Python-3.8.10 torch-1.13.0+cu117 CPU

Fusing layers... 
Model summary: 261 layers, 61497430 parameters, 0 gradients, 154.5 GFLOPs
Adding AutoShape... 


In [9]:
tablas_1 = retinanet_tablas([img_tensor])
valid_idx_1 = torchvision.ops.nms(tablas_1[1][0]['boxes'], tablas_1[1][0]['scores'], 0.1)

coords_1 = tablas_1[1][0]['boxes'][valid_idx_1]
scores_1 = tablas_1[1][1]['scores'][valid_idx_1]

IndexError: list index out of range

In [ ]:
tablas_2 = faster_rcnn_tablas([img_tensor])
valid_idx_2 = torchvision.ops.nms(tablas_2[1][0]['boxes'], tablas_2[1][0]['scores'], 0.1)

coords_2 = tablas_2[1][0]['boxes'][valid_idx_1]
scores_2 = tablas_2[1][1]['scores'][valid_idx_1]

In [ ]:
tablas_3 = yolo_tablas(Image.open('/home/nacho/TFI-Cazcarra/data/imagenes_diagramas/ERDiagramsMySQL-1.png').convert("RGB"))

#x1 y1 x2 y2 score class
coords_3 = tablas_3.xyxyn[0][:, :4]
scores_3 = tablas_3.xyxyn[0][:, 4]

valid_idx_3 = torchvision.ops.nms(coords_3, scores_3, 0.1)

# Falta filtrar acá

### 2. Extraigo las cardinalidades

In [ ]:
retinanet_tablas = load_model(os.path.join(PATH_MODELS, "model_best_cardinalidades_retinanet.pt"))
faster_rcnn_tablas = load_model(os.path.join(PATH_MODELS, "model_best_cardinalidades_fasterrcnn.pt"))

yolo_tablas = torch.hub.load('ultralytics/yolov5', 'custom', \
                             os.path.join(PATH_YOLO, "exp3", "weights", "best.pt"))

In [ ]:
## TODO: Dividir la imagen en frames y etc etc.

## Encuentro las líneas y junto con las cardinalidades

In [10]:
OFFSET_TABLAS = 5

In [11]:
def find_lines(img_path):
    img_basename = os.path.basename(img_path)
    tablas, cardinalidades = get_tablas(img_basename)
    #Agrandamos un toque las cajas
    offset = np.array([-OFFSET_TABLAS, -OFFSET_TABLAS, OFFSET_TABLAS, OFFSET_TABLAS]).reshape(1,4)
    tablas = np.sum([tablas, offset])

    #No le paso las cardinalidades.
    img, all_lines = apply_hough(img_path, tablas, [])

    all_points = lines_to_points(all_lines)
    lines = hough_detecting(all_points)

    return unify_cardinalidades(img, lines, cardinalidades)

In [12]:
def line_to_points(x, y, max_dst_per_points=2):
    dst_line = math.dist(x,y)
    parts = dst_line // max_dst_per_points
    return np.linspace(x, y, int(parts)+1).tolist()


def dist_func(comb):
    return math.dist(comb[0], comb[1])


def get_centroid(cardinalidad):
    center_x  = cardinalidad[0] + (cardinalidad[2] - cardinalidad[0])/2
    center_y  = cardinalidad[1] + (cardinalidad[3] - cardinalidad[1])/2
    return (center_x, center_y)

def nearest_tabla_from_cardinalidad(cardinalidad, tablas):
    # Dadas las coords de una cardinalidad, devolver a qué tabla está más cerca.
    # TODO: Puedo mezclarlo con el de OCR y reemplazar las coordenadas por el nombre de la
    # tabla... asi se entiende mejor. Esto quizás no hace falta pero no estaría mal.
    cardinalidad = literal_eval(cardinalidad)
    cardinalidad_centroid = get_centroid(cardinalidad)
    dict_dist_tablas = {}
    for t in tablas:
        tabla_points = []
        # De x1y1 a x2y1
        tabla_points += line_to_points((t[0], t[1]), (t[2], t[1]))
        # De x1y1 a x1y2
        tabla_points += line_to_points((t[0], t[1]), (t[0], t[3]))
        # De x1y2 a x2y2
        tabla_points += line_to_points((t[0], t[3]), (t[2], t[3]))
        # De x2y1 a x2y2
        tabla_points += line_to_points((t[2], t[1]), (t[2], t[3]))
        dist_cardinalidad = [(cardinalidad_centroid, point) for point in tabla_points]
        min_combination = min(dist_cardinalidad, key=dist_func)
        dict_dist_tablas[",".join([str(c) for c in t])] = dist_func(min_combination)
    nearest_tabla = min(dict_dist_tablas, key=dict_dist_tablas.get)
    return [int(c) for c in nearest_tabla.split(",")]

In [13]:
pairs = []

img_basename = os.path.basename(img_path)
tablas, _ = get_tablas(img_basename)

for line_name, line in find_lines(img_path).items():
    cardinalidades = line.split("|")
    tabla_a = nearest_tabla_from_cardinalidad(cardinalidades[0], tablas)
    tabla_b = nearest_tabla_from_cardinalidad(cardinalidades[1], tablas)
    pairs.append((tabla_a, tabla_b))

In [14]:
pairs

[([289, 16, 440, 150], [18, 89, 211, 344]),
 ([291, 191, 449, 383], [18, 89, 211, 344]),
 ([291, 191, 449, 383], [516, 241, 664, 334])]

## OCR

In [15]:
# Una vez que tenemos la lista de tuplas con las conexiones de las tablas, podemos sacar
# las FK y de ahi, descartar los demás y poder sacar la/s PK.

In [16]:
# TODO: Choose algo and language.
ocr = PaddleOCR(use_angle_cls=False, lang='en', show_log=False, 
                det_algorithm_dir="/home/nacho/TFI-Cazcarra/ocr/db/", 
                rec_algorithm_dir="/home/nacho/TFI-Cazcarra/ocr/svtr/", det_db_score_mode="slow") 

In [17]:
coords = []
results = []

img_arr = np.array(img)

for t in tablas:
    tabla_cropped = img_arr[t[1]:t[3], t[0]:t[2]]
    tabla_cropped = reescale(tabla_cropped)
    if len(tabla_cropped.shape) == 3 and tabla_cropped.shape[-1] > 3:
        tabla_cropped = tabla_cropped[:,:,:3]
    result = ocr.ocr(tabla_cropped, cls=False)
    
    coords.append(t.tolist())
    results.append(result[0])

In [18]:
all_tables = {}
tables_names = {}

for c, r in zip(coords, results):
    boxes = [line[0] for line in r]
    txts = [line[1][0].strip() for line in r]
    scores = [line[1][1] for line in r]
    table, dict_attributes = clean_texts(txts)
    all_tables[table] = dict_attributes
    tables_names[table] = c

In [19]:
def pairs_to_names(pairs, tables_names):
    new_pairs = []
    tables_names = {str(v):k for k,v in tables_names.items()}
    
    for pair in pairs:
        tabla_a, tabla_b = pair
        tabla_a, tabla_b = tables_names[str(tabla_a)], tables_names[str(tabla_b)]
        new_pairs.append((tabla_a, tabla_b))
    return new_pairs

In [29]:
pairs = pairs_to_names(pairs, tables_names)

In [26]:
def extract_candidate_keys(table_attributes):
    return [attr for attr in table_attributes if "_id" in attr]

In [143]:
def get_primary_keys(candidates, fks):
    return list(set(candidates) - set(fks))

def get_foreign_keys(table, candidates, pairs, lemmatizer=nlp_english):
    def is_foreign_key(table_pair, candidates, table, lemmatizer):
        pair = table_pair + "_id"
        
        table_pair_lemmatized = lemmatizer(table_pair)[0].lemma_
        pair_lemmatized = table_pair_lemmatized + "_id"
        
        table_match = (table_pair == table or table_pair_lemmatized == table)
        
        pair_match = (pair in candidates and not table_match)
        pair_lemmatized_match = (pair_lemmatized in candidates and not table_match)
                
        if pair_match:
            return True, pair
        elif pair_lemmatized_match:
            return True, pair_lemmatized
        else:
            return False, ""
    """
    Ejemplo:
    table -> poems
    candidates -> ['poems_id', 'users_id', 'categories_id']
    pairs -> [('tokens', 'users'), ('poems', 'users'), ('poems', 'categories')]
    """
    fks = {}
    for pair in pairs:
        if table not in pair:
            continue
        is_fk_pair0, p0 = is_foreign_key(table_pair=pair[0], candidates=candidates, \
                                         table=table, lemmatizer=lemmatizer)
        is_fk_pair1, p1 = is_foreign_key(table_pair=pair[1], candidates=candidates, \
                                         table=table, lemmatizer=lemmatizer)
        if is_fk_pair0:
            fks[p0] = pair[0]
        elif is_fk_pair1:
            fks[p1] = pair[1]
    return fks

In [146]:
## TODO: En base a esto construir las FK y las PK. Una vez hecho eso, también definir las many to many.
for k in all_tables.keys():  
    candidates = extract_candidate_keys(all_tables[k].keys())
    fks = get_foreign_keys(table=k, candidates=candidates, pairs=pairs)
    pks = {pk: k for pk in get_primary_keys(candidates, fks.keys())}
    print(f"Table: {k}")
    print(f"Candidates: {candidates}")
    print(f"FKS: {fks}")
    print(f"PKS: {pks}")
    print("####################")

Table: users
Candidates: ['user_id']
FKS: {}
PKS: {'user_id': 'users'}
####################
Table: poems
Candidates: ['poem_id', 'category_id', 'user_id']
FKS: {'user_id': 'users', 'category_id': 'categories'}
PKS: {'poem_id': 'poems'}
####################
Table: tokens
Candidates: ['token_id', 'user_id']
FKS: {'user_id': 'users'}
PKS: {'token_id': 'tokens'}
####################
Table: categories
Candidates: ['category_id']
FKS: {}
PKS: {'category_id': 'categories'}
####################


In [244]:
def generate_pks_code(pks):
    keys = pks.keys()
    keys = ", ".join(keys)
    return f"PRIMARY KEY ({keys})"

def generate_fks_code(fks):
    code = ""
    for fk, table_reference in fks.items():
        code += f"FOREIGN KEY ({fk}) REFERENCES {table_reference}({fk}),\n   "
    return code[:-5]

In [245]:
def create_code(table, dict_attributes, primary_keys, foreign_keys):
    '''
    Crea una tabla de MySQL
    '''
    attributes_code = "  "
    i = 0
    for k, v in dict_attributes.items():
        attributes_code += k + " " + v           
        attributes_code += ",\n   "
        i += 1
    pks_code = generate_pks_code(primary_keys)
    fks_code = generate_fks_code(foreign_keys)
    if pks_code:
        attributes_code += pks_code
        attributes_code += ",\n   " if fks_code else ""
    if fks_code:
        attributes_code += fks_code
    code = f" CREATE TABLE {table} ( \n {attributes_code} \n );"
    return code

In [246]:
for k, dict_attributes in all_tables.items():
    candidates = extract_candidate_keys(all_tables[k].keys())
    fks = get_foreign_keys(table=k, candidates=candidates, pairs=pairs)
    pks = {pk: k for pk in get_primary_keys(candidates, fks.keys())}
    print(create_code(k, dict_attributes, pks, fks))

 CREATE TABLE users ( 
   user_id INT(11),
   first_name VARCHAR(50),
   last_name VARCHAR(50),
   email VARCHAR(100),
   username VARCHAR(30),
   pass_phrase VARCHAR(500),
   is_admin TINYINT(4),
   date_registered DATETIME,
   profile_pic VARCHAR(30),
   registration_confirmed TINYINT(4),
   PRIMARY KEY (user_id) 
 )
 CREATE TABLE poems ( 
   poem_id INT(11),
   title VARCHAR(200),
   poem TEXT,
   date_submitted DATETIME,
   category_id INT(11),
   user_id INT(11),
   date_approved DATETIME,
   PRIMARY KEY (poem_id),
   FOREIGN KEY (user_id) REFERENCES users(user_id),
   FOREIGN KEY (category_id) REFERENCES categories(category_id) 
 )
 CREATE TABLE tokens ( 
   token_id INT(11),
   token CHAR(64),
   user_id INT(11),
   token_expires DATETIME,
   PRIMARY KEY (token_id),
   FOREIGN KEY (user_id) REFERENCES users(user_id) 
 )
 CREATE TABLE categories ( 
   category_id INT(11),
   category VARCHAR(100),
   PRIMARY KEY (category_id) 
 )


In [247]:
import spacy

nlp_english = spacy.load("en_core_web_sm")
nlp_spanish = spacy.load("es_core_news_sm")

In [98]:
[word.lemma_ for word in nlp_english("users_cars")]

['users_car']

In [97]:
[word.lemma_ for word in nlp_spanish("ventanales")]

['ventanal']

In [80]:
# !python -m spacy download en_core_web_sm

In [81]:
# !python -m spacy download es_core_news_sm

## Una vez que está todo hecho, unificar acá y detectar puntos de mejora.